In [ ]:
!pip install torch torchvision

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 57.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 45.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 83.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
import torch
import torch.nn as nn

class YOLOv1(nn.Module):
    def __init__(self, S=7, B=2, C=20):
        super(YOLOv1, self).__init__()
        self.S = S
        self.B = B
        self.C = C

        self.conv_layers = nn.Sequential(
            nn.Conv2d(3, 64, 7, 2, 3), nn.LeakyReLU(0.1),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(64, 192, 3, 1, 1), nn.LeakyReLU(0.1),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(192, 128, 1), nn.LeakyReLU(0.1),
            nn.Conv2d(128, 256, 3, 1, 1), nn.LeakyReLU(0.1),
            nn.Conv2d(256, 256, 1), nn.LeakyReLU(0.1),
            nn.Conv2d(256, 512, 3, 1, 1), nn.LeakyReLU(0.1),
            nn.MaxPool2d(2, 2),
            *[nn.Sequential(
                nn.Conv2d(512, 256, 1), nn.LeakyReLU(0.1),
                nn.Conv2d(256, 512, 3, 1, 1), nn.LeakyReLU(0.1),
            ) for _ in range(4)],
            nn.Conv2d(512, 512, 1), nn.LeakyReLU(0.1),
            nn.Conv2d(512, 1024, 3, 1, 1), nn.LeakyReLU(0.1),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(1024, 512, 1), nn.LeakyReLU(0.1),
            nn.Conv2d(512, 1024, 3, 1, 1), nn.LeakyReLU(0.1),
            nn.Conv2d(1024, 512, 1), nn.LeakyReLU(0.1),
            nn.Conv2d(512, 1024, 3, 1, 1), nn.LeakyReLU(0.1),
            nn.Conv2d(1024, 1024, 3, 2, 1), nn.LeakyReLU(0.1),
            nn.Conv2d(1024, 1024, 3, 1, 1), nn.LeakyReLU(0.1),
            nn.Conv2d(1024, 1024, 3, 1, 1), nn.LeakyReLU(0.1),
        )

        self.fc_layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(1024 * 7 * 7, 4096),
            nn.Dropout(0.5),
            nn.LeakyReLU(0.1),
            nn.Linear(4096, S * S * (C + B * 5))
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = self.fc_layers(x)
        x = x.view(-1, self.S, self.S, self.C + self.B * 5)
        return x

class YoloLoss(nn.Module):
    def __init__(self, S=7, B=2, C=20, lambda_coord=5, lambda_noobj=0.5):
        super(YoloLoss, self).__init__()
        self.mse = nn.MSELoss(reduction="sum")
        self.S = S
        self.B = B
        self.C = C
        self.lambda_coord = lambda_coord
        self.lambda_noobj = lambda_noobj

    def forward(self, predictions, target):
        N = predictions.size(0)
        coord_mask = target[..., 4] > 0
        noobj_mask = target[..., 4] == 0

        coord_pred = predictions[coord_mask]
        coord_target = target[coord_mask]

        coord_loss = self.mse(coord_pred[..., 0], coord_target[..., 0]) + \
                     self.mse(coord_pred[..., 1], coord_target[..., 1]) + \
                     self.mse(torch.sqrt(torch.abs(coord_pred[..., 2]) + 1e-6), torch.sqrt(torch.abs(coord_target[..., 2]) + 1e-6)) + \
                     self.mse(torch.sqrt(torch.abs(coord_pred[..., 3]) + 1e-6), torch.sqrt(torch.abs(coord_target[..., 3]) + 1e-6))

        obj_loss = self.mse(coord_pred[..., 4], coord_target[..., 4])

        noobj_loss = self.mse(predictions[noobj_mask][..., 4], target[noobj_mask][..., 4])

        class_loss = self.mse(coord_pred[..., 5:], coord_target[..., 5:])

        total_loss = self.lambda_coord * coord_loss + obj_loss + \
                     self.lambda_noobj * noobj_loss + class_loss

        return total_loss / N


In [ ]:
model = YOLOv1()
loss_fn = YoloLoss()

# Create dummy inputs and targets
x = torch.randn((2, 3, 448, 448))  # batch of 2 images
output = model(x)  # model prediction

target = torch.randn_like(output)  # dummy target
loss = loss_fn(output, target)  # compute loss

print(f"Output shape: {output.shape}")
print(f"Loss: {loss.item()}")


Output shape: torch.Size([2, 7, 7, 30])
Loss: 906.2088623046875
